In [8]:
import pandas
from sklearn.model_selection import train_test_split as tts

dota_data = pandas.read_csv('dota2Train.csv', header = None)

In [9]:
(training_data, validation_data) = tts(dota_data)